# 实验 3.1：FGSM 白盒对抗攻击

## 🎯 学习目标

完成本实验后，你将能够：
- 解释 FGSM（快速梯度符号法）的攻击原理
- 实现基于梯度的对抗样本生成方法
- 分析扰动强度对攻击效果和隐蔽性的影响
- 理解白盒攻击的威胁及其在真实场景中的意义

## 📚 前置知识

- 理解神经网络的前向传播和反向传播
- 了解梯度和损失函数的概念
- 完成模块三理论章节：《对抗样本基础》

## 🖥️ 实验环境

- 平台：腾讯 Cloud Studio（https://cloudstudio.net/）
- GPU：NVIDIA Tesla T4（16GB 显存）
- 模型：ResNet-18（ImageNet 预训练）
- 框架：PyTorch 2.0+

## 📝 填空说明

本实验共 **4 个填空练习**，难度：⭐⭐⭐☆☆

## 第一部分：环境准备与依赖安装

In [ ]:
# 安装必要的依赖（首次运行需要几分钟）
%pip install torch torchvision -q

# 导入必要的库
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models, transforms
from PIL import Image

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子以确保实验可重复
torch.manual_seed(42)
np.random.seed(42)

# 检查环境
print("=" * 50)
print("🔧 环境检查")
print("=" * 50)
print(f"  PyTorch 版本: {torch.__version__}")
print(f"  CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU 设备: {torch.cuda.get_device_name(0)}")
print("✓ 环境准备完成！")
print("=" * 50)

In [ ]:
# 加载预训练的 ResNet-18 模型
# ResNet-18 是一个在 ImageNet 数据集上训练的图像分类模型
# ImageNet 包含 1000 个类别的日常物体图像

print("正在加载预训练模型...")
model = models.resnet18(pretrained=True)
model.eval()  # 设置为评估模式（关闭 Dropout 和 BatchNorm 的训练行为）

# 统计模型参数
total_params = sum(p.numel() for p in model.parameters())
print(f"✓ ResNet-18 模型加载成功！")
print(f"  参数量: {total_params:,} (~11M)")
print(f"  输出类别: 1000 (ImageNet 类别)")

# ImageNet 类别标签（部分常见类别，用于演示）
IMAGENET_LABELS = {
    0: "蝾螈", 1: "金鱼", 2: "大白鲨", 3: "虎鲨", 
    7: "公鸡", 8: "母鸡", 9: "鸵鸟", 
    281: "虎斑猫", 282: "虎", 285: "埃及猫", 291: "狮子",
    388: "大熊猫", 389: "小熊猫",
    417: "气球", 433: "浴缸", 
    504: "咖啡杯", 605: "iPod", 
    852: "网球", 920: "盘子"
}

print(f"  已加载 {len(IMAGENET_LABELS)} 个常见类别标签")

In [ ]:
# 定义图像预处理和辅助函数

# ImageNet 模型需要特定的输入格式和归一化
preprocess = transforms.Compose([
    transforms.Resize(256),           # 缩放到 256x256
    transforms.CenterCrop(224),       # 中心裁剪到 224x224
    transforms.ToTensor(),            # 转换为张量，范围 [0, 1]
])

# ImageNet 标准化参数（根据训练集统计得出）
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],  # RGB 三通道的均值
    std=[0.229, 0.224, 0.225]     # RGB 三通道的标准差
)

def predict(model, img_tensor):
    """
    使用模型对图像进行预测
    
    参数:
        model: 预训练模型
        img_tensor: 图像张量 [C, H, W]，范围 [0, 1]
    
    返回:
        pred_class: 预测类别（整数）
        confidence: 预测置信度（概率）
    """
    # 步骤1：归一化图像
    input_tensor = normalize(img_tensor).unsqueeze(0)  # 添加 batch 维度 [1, C, H, W]
    
    # 步骤2：前向传播
    with torch.no_grad():  # 不计算梯度，节省内存
        output = model(input_tensor)
    
    # 步骤3：计算概率（使用 softmax）
    probs = torch.softmax(output, dim=1)
    
    # 步骤4：获取预测类别和置信度
    pred_class = output.argmax(dim=1).item()
    confidence = probs[0, pred_class].item()
    
    return pred_class, confidence

print("✓ 辅助函数定义完成！")

## 第二部分：准备测试图像并观察原始预测

为了实验的可重复性和教学清晰度，我们生成一个合成测试图像。这样可以确保每次运行实验都得到相同的结果。

In [ ]:
# 创建一个合成测试图像
def create_test_image():
    """
    创建一个用于测试的合成图像
    图案：中心黑色圆形，外围白色边缘，背景为灰色
    """
    # 创建 224x224 的随机图像背景
    np.random.seed(42)  # 固定随机种子确保可重复性
    img = np.random.rand(224, 224, 3) * 0.3 + 0.35  # 灰色背景
    
    # 绘制中心圆形图案（模拟某种物体）
    center_x, center_y = 112, 112
    for i in range(224):
        for j in range(224):
            dist = np.sqrt((i - center_x)**2 + (j - center_y)**2)
            if dist < 60:
                img[i, j] = [0.1, 0.1, 0.1]  # 黑色中心
            elif dist < 80:
                img[i, j] = [0.9, 0.9, 0.9]  # 白色边缘
    
    # 转换为 PyTorch 张量格式 [C, H, W]
    img_tensor = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)
    return img_tensor

# 创建测试图像
original_image = create_test_image()

# 可视化原始图像
plt.figure(figsize=(5, 5))
plt.imshow(original_image.permute(1, 2, 0).numpy())
plt.title("原始测试图像", fontsize=14, pad=10)
plt.axis('off')
plt.tight_layout()
plt.show()

# 获取模型的原始预测
original_class, original_confidence = predict(model, original_image)
original_label = IMAGENET_LABELS.get(original_class, f"未知类别")

print("=" * 50)
print("📊 原始预测结果")
print("=" * 50)
print(f"  预测类别: {original_label} (ID: {original_class})")
print(f"  预测置信度: {original_confidence:.2%}")
print("=" * 50)

### ✅ 检查点 1

运行到这里，你应该看到：
- ✓ 一张合成的测试图像（中心黑色圆形，白色边缘）
- ✓ 模型对该图像的预测结果和置信度

如果遇到问题，请检查：
- 模型是否成功加载
- 图像是否正确显示

## 第三部分：实现 FGSM 对抗攻击

现在我们来实现 FGSM 攻击的核心算法。关键步骤包括：
1. 计算模型对输入的梯度
2. 获取梯度的符号
3. 生成对抗扰动
4. 添加扰动得到对抗样本

In [ ]:
def fgsm_attack(model, image, label, epsilon):
    """
    实现 FGSM (Fast Gradient Sign Method) 对抗攻击
    
    参数:
        model: 目标神经网络模型
        image: 原始图像张量 [C, H, W]，范围 [0, 1]
        label: 原始图像的真实标签（整数）
        epsilon: 扰动强度参数 ε
    
    返回:
        adversarial_image: 对抗样本张量 [C, H, W]
    """
    # 步骤1：复制图像并启用梯度追踪
    # 我们需要对输入图像计算梯度，所以必须设置 requires_grad=True
    image_copy = image.clone().detach()
    image_copy.requires_grad = True
    
    # 步骤2：归一化并前向传播
    # 添加 batch 维度 [1, C, H, W]
    normalized = normalize(image_copy).unsqueeze(0)
    output = model(normalized)
    
    # 步骤3：计算损失
    # 使用交叉熵损失，目标是最大化正确类别的损失
    loss = nn.CrossEntropyLoss()(output, torch.tensor([label]))
    
    # 步骤4：反向传播计算梯度
    model.zero_grad()  # 清空模型的梯度
    loss.backward()    # 计算梯度
    
    # 步骤5：获取输入图像的梯度
    # gradient 的形状与 image_copy 相同
    gradient = image_copy.grad.data
    
    # ========== 填空 1：生成对抗扰动 ==========
    # 
    # 🎯 任务：根据 FGSM 公式计算对抗扰动
    # 
    # 💡 提示：
    #   - FGSM 公式：perturbation = ε × sign(梯度)
    #   - sign() 函数返回梯度的符号（+1 或 -1）
    #   - 使用 torch.sign() 函数
    # 
    # 📖 参考文档：
    #   torch.sign(tensor) 返回张量中每个元素的符号
    # 
    # 难度：⭐⭐☆☆☆
    
    perturbation = ___________  # 期望：epsilon * torch.sign(gradient)
    
    # 步骤6：生成对抗样本
    adversarial_image = image_copy + perturbation
    
    # 步骤7：裁剪到有效像素范围 [0, 1]
    # 确保生成的对抗样本是有效图像
    adversarial_image = torch.clamp(adversarial_image, 0, 1)
    
    return adversarial_image.detach()

print("✓ FGSM 攻击函数定义完成！")

In [ ]:
# ========== 填空 2：执行 FGSM 攻击 ==========
# 
# 🎯 任务：调用 fgsm_attack 函数生成对抗样本
# 
# 💡 提示：
#   - 需要传入：model（模型）、original_image（原图）、original_class（标签）、epsilon（扰动强度）
#   - epsilon 建议从 0.03 开始尝试
#   - 函数返回对抗样本
# 
# 难度：⭐☆☆☆☆

epsilon = 0.03  # 扰动强度参数

# 执行攻击
adversarial_image = ___________  # 期望：fgsm_attack(model, original_image, original_class, epsilon)

# 获取对抗样本的预测结果
adv_class, adv_confidence = predict(model, adversarial_image)
adv_label = IMAGENET_LABELS.get(adv_class, f"未知类别")

# 显示攻击结果
print("=" * 50)
print("🎯 FGSM 攻击结果")
print("=" * 50)
print(f"  原始预测: {original_label} (ID: {original_class}) - 置信度: {original_confidence:.2%}")
print(f"  攻击后预测: {adv_label} (ID: {adv_class}) - 置信度: {adv_confidence:.2%}")
print(f"  扰动强度 ε: {epsilon}")

# 判断攻击是否成功
if original_class != adv_class:
    print(f"  攻击结果: ✓ 成功！模型预测发生改变")
else:
    print(f"  攻击结果: ✗ 失败，模型预测未改变")
print("=" * 50)

### 🤔 思考一下

1. **观察现象**：对抗样本的预测结果与原图有什么不同？
2. **思考原理**：为什么添加一个很小的扰动就能让模型改变预测？
3. **安全启示**：这种攻击在现实世界中意味着什么威胁？

## 第四部分：可视化对抗样本和扰动

对抗攻击的一个重要特性是**对人眼不可见**。让我们可视化原图、扰动和对抗样本，观察它们的差异。

In [ ]:
# ========== 填空 3：计算对抗扰动 ==========
# 
# 🎯 任务：计算对抗样本与原图的差异（扰动）
# 
# 💡 提示：
#   - 扰动 = 对抗样本 - 原始图像
#   - 这是一个简单的减法操作
# 
# 难度：⭐☆☆☆☆

perturbation = ___________  # 期望：adversarial_image - original_image

# 可视化对比
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 子图1：原始图像
axes[0].imshow(original_image.permute(1, 2, 0).numpy())
axes[0].set_title(f"原始图像\n预测: {original_label}\n置信度: {original_confidence:.1%}", 
                  fontsize=12, color='green')
axes[0].axis('off')

# 子图2：对抗扰动（放大显示以便观察）
# 将扰动归一化到 [0, 1] 范围以便可视化
perturbation_vis = perturbation - perturbation.min()
perturbation_vis = perturbation_vis / (perturbation_vis.max() + 1e-8)
axes[1].imshow(perturbation_vis.permute(1, 2, 0).numpy())
axes[1].set_title(f"对抗扰动（放大显示）\nε = {epsilon}", 
                  fontsize=12, color='orange')
axes[1].axis('off')

# 子图3：对抗样本
axes[2].imshow(adversarial_image.permute(1, 2, 0).numpy())
attack_color = 'red' if original_class != adv_class else 'blue'
axes[2].set_title(f"对抗样本\n预测: {adv_label}\n置信度: {adv_confidence:.1%}", 
                  fontsize=12, color=attack_color)
axes[2].axis('off')

plt.tight_layout()
plt.show()

# 计算并显示扰动的统计信息
print("=" * 50)
print("📏 扰动统计分析")
print("=" * 50)
print(f"  最大扰动值: {perturbation.abs().max().item():.6f}")
print(f"  平均扰动值: {perturbation.abs().mean().item():.6f}")
print(f"  扰动范围: [{perturbation.min().item():.6f}, {perturbation.max().item():.6f}]")
print(f"  L∞ 范数: {perturbation.abs().max().item():.6f} (应接近 ε={epsilon})")
print("=" * 50)

## 第五部分：探索扰动强度的影响

**核心问题**：扰动强度 ε 如何影响攻击效果和图像质量？

我们将测试不同的 ε 值，观察：
1. **攻击成功率**：ε 越大，攻击越容易成功吗？
2. **图像质量**：ε 越大，图像变化越明显吗？
3. **平衡点**：是否存在一个最优的 ε 值？

In [ ]:
# 测试不同扰动强度的效果
epsilon_values = [0.0, 0.01, 0.02, 0.03, 0.05, 0.1]

print("=" * 70)
print("🔬 不同 ε 值的攻击效果对比")
print("=" * 70)
print(f"{'ε 值':<10} {'预测类别':<15} {'置信度':<12} {'攻击结果':<15}")
print("-" * 70)

results = []
for eps in epsilon_values:
    if eps == 0:
        # ε=0 时就是原图，不需要攻击
        test_pred, test_conf = original_class, original_confidence
        test_img = original_image
    else:
        # ========== 填空 4：对不同 epsilon 执行攻击 ==========
        # 
        # 🎯 任务：对每个 epsilon 值执行 FGSM 攻击
        # 
        # 💡 提示：
        #   - 调用 fgsm_attack 函数
        #   - 参数：model, original_image, original_class, eps
        #   - 将结果存储在 test_img 变量中
        # 
        # 难度：⭐⭐☆☆☆
        
        test_img = ___________  # 期望：fgsm_attack(model, original_image, original_class, eps)
        test_pred, test_conf = predict(model, test_img)
    
    # 判断攻击是否成功
    success = "✓ 成功" if test_pred != original_class else "✗ 失败"
    test_label = IMAGENET_LABELS.get(test_pred, f"类别{test_pred}")
    
    results.append((eps, test_img, test_pred, test_conf, success))
    print(f"{eps:<10.2f} {test_label:<15} {test_conf:<12.2%} {success:<15}")

print("=" * 70)

In [ ]:
# 可视化不同 ε 值的对抗样本
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (eps, img, pred, conf, success) in enumerate(results):
    axes[idx].imshow(img.permute(1, 2, 0).numpy())
    
    # 根据攻击结果设置颜色
    color = 'red' if pred != original_class else 'green'
    label = IMAGENET_LABELS.get(pred, f"类别{pred}")
    
    axes[idx].set_title(
        f"ε = {eps}\n预测: {label}\n置信度: {conf:.1%}\n{success}",
        fontsize=11,
        color=color
    )
    axes[idx].axis('off')

plt.suptitle("不同扰动强度 ε 的对抗样本效果对比", fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

### 🤔 深入思考

基于上面的实验结果，思考以下问题：

1. **扰动与成功率**：从 ε=0.01 到 ε=0.1，攻击成功率如何变化？
2. **扰动与隐蔽性**：哪个 ε 值既能成功攻击，又不会让图像变化太明显？
3. **实际应用**：在现实场景中（如人脸识别、自动驾驶），攻击者会如何选择 ε？
4. **防御思路**：如果你是防御者，如何检测或防御这种攻击？

## 📋 实验小结

### 核心收获

通过本实验，你已经：

1. **理解原理**：掌握了 FGSM 对抗攻击的数学原理和实现方法
2. **动手实践**：成功生成了第一个对抗样本，观察了攻击效果
3. **参数探索**：分析了扰动强度 ε 对攻击成功率和隐蔽性的影响
4. **安全认知**：认识到深度学习模型在面对对抗样本时的脆弱性

### 关键代码回顾

```python
# FGSM 核心公式
x_adv = x + ε × sign(∇_x Loss(θ, x, y))

# PyTorch 实现
perturbation = epsilon * torch.sign(gradient)
adversarial_image = original_image + perturbation
```

### 🔑 关键要点

- **白盒攻击**：攻击者需要知道模型结构和参数
- **梯度方向**：沿着损失函数梯度的符号方向添加扰动
- **参数权衡**：ε 越大攻击越强，但扰动越明显
- **L∞ 约束**：FGSM 保证每个像素的扰动不超过 ε

### 实际应用场景

FGSM 攻击在以下场景中具有现实威胁：
- **图像分类系统**：欺骗内容审核、垃圾邮件过滤
- **人脸识别**：绕过身份验证、隐私保护
- **自动驾驶**：干扰交通标志识别
- **医疗诊断**：影响疾病诊断准确性

### 延伸学习

- **下一个实验**：实验 3.2 - PGD 迭代攻击（更强的对抗攻击方法）
- **论文阅读**：Goodfellow et al., "Explaining and Harnessing Adversarial Examples" (ICLR 2015)
- **进阶挑战**：尝试实现 Targeted FGSM（让模型预测为指定类别）

## 📖 参考答案

<details>
<summary>点击展开参考答案</summary>

**填空 1：生成对抗扰动**
```python
perturbation = epsilon * torch.sign(gradient)
```

**填空 2：执行 FGSM 攻击**
```python
adversarial_image = fgsm_attack(model, original_image, original_class, epsilon)
```

**填空 3：计算对抗扰动**
```python
perturbation = adversarial_image - original_image
```

**填空 4：对不同 epsilon 执行攻击**
```python
test_img = fgsm_attack(model, original_image, original_class, eps)
```

</details>

In [ ]:
# 实验结束，清理显存（可选）
# 如果要继续进行下一个实验，可以运行这段代码释放显存

del model
import torch
torch.cuda.empty_cache()
print("✓ 显存已清理")

⚠️ 重要提醒

实验完成后，请记得：

1. **释放 GPU 资源**：如果你使用的是 Tencent CloudStudio, 请点击右上角的 "停止" 按钮，停止运行。
2. **保存实验结果**：如需保存，请下载 notebook 文件

这样可以避免不必要的资源占用和费用产生。**否则会一直消耗你的免费资源额度。**